#### Neural Transition-Based Parser

We will now train an `oracle` model for a greedy (arc-standard) transition based-parser. The model extracts features from the state of a parse and uses these features to predict a probability distribution over all possible next actions. For unlabeled arcs, we only have three possible actions: `LEFTARC`, `RIGHTARC` and `SHIFT`. The model architecture is shown below (diagram borrowed from Jurafsky-Martin textbook):

<img src="neural_oracle.png" width="550" height="320">

The most important features needed to predict the next action are usually the top few words on the stack and buffer (and dependents of these words), which is why for this model, we will construct a simple feature vector by concatenating the contextualized embeddings (from A BERT encoder) of the top two words from the stack and the top word from the buffer. We will designate the BERT `[CLS]` token as the `ROOT` and we will use the zero vector to represent `NULL` (e.g. if the buffer is empty or the stack con tains less than 2 words, we fill the empty positions with NULL).

The feature vector is then passed through a basic 2-layer `feed-forward network` with a softmax at the output to obtain the predicted probability distribution over actions. In order to accomodate labelled arcs, we have two options: 

1) Augment the action with the label, i.e. for each `label`, we have two actions: `LEFTARC-label`, `RIGHTARC-label`

2) Create a separate feed-forward network which predicts the label, independent from the action. Since the `SHIFT` action has no associated label, we will define a special `NULL label` that goes along with it.

Option 2 makes the learning task easier (because there are fewer class labels to predict) and is more efficient and could potentially lead to better performance (since it requires fewer parameters and so there's less chance of overfitting).
